Vamos a partir de lo que hemos visto en el notebook anterior...

# Lectura de un fichero de datos

In [7]:
# primero hacemos los imports de turno
import os
import datetime as dt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# Leemos los datos del fichero 'mast.txt'
ipath = os.path.join('Datos', 'mast.txt')

def dateparse(date, time):
    YY = 2000 + int(date[:2])
    MM = int(date[2:4])
    DD = int(date[4:])
    hh = int(time[:2])
    mm = int(time[2:])
    
    return dt.datetime(YY, MM, DD, hh, mm, 0)
    

cols = ['Date', 'time', 'wspd', 'wspd_max', 'wdir',
        'x1', 'x2', 'x3', 'x4', 'x5', 
        'wspd_std']
wind = pd.read_csv(ipath, sep = "\s*", names = cols, 
                   parse_dates = [[0, 1]], index_col = 0,
                   date_parser = dateparse)

/home/tornado/miniconda3/envs/sciconda/lib/python3.5/site-packages/ipykernel/__main__.py:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
/home/tornado/miniconda3/envs/sciconda/lib/python3.5/site-packages/pandas/io/parsers.py:1588: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/tornado/miniconda3/envs/sciconda/lib/python3.5/site-packages/pandas/io/parsers.py:1590: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


# Información básica (en este caso de un `DataFrame`)

In [9]:
wind.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100886 entries, 2013-09-04 00:00:00 to 2015-09-03 12:00:00
Data columns (total 9 columns):
wspd        100886 non-null float64
wspd_max    100886 non-null float64
wdir        100886 non-null float64
x1          100886 non-null float64
x2          100886 non-null float64
x3          100886 non-null float64
x4          100886 non-null float64
x5          100886 non-null float64
wspd_std    100886 non-null float64
dtypes: float64(9)
memory usage: 7.7 MB


In [10]:
wind.describe()

,wspd,wspd_max,wdir,x1,x2,x3,x4,x5,wspd_std
count,100886.000000,100886.000000,100886.000000,100886.00,100886.00,1.008860e+05,1.008860e+05,1.008860e+05,100886.000000
mean,10.062296,11.508717,200.397350,999.99,999.99,9.999000e+01,9.999990e+03,9.999990e+03,0.555674
std,4.819671,5.524473,92.479215,0.00,0.00,1.421093e-14,1.818998e-12,1.818998e-12,0.351963
min,0.020000,0.140000,0.000000,999.99,999.99,9.999000e+01,9.999990e+03,9.999990e+03,0.010000
25%,6.440000,7.420000,142.100000,999.99,999.99,9.999000e+01,9.999990e+03,9.999990e+03,0.310000
50%,9.640000,10.890000,213.200000,999.99,999.99,9.999000e+01,9.999990e+03,9.999990e+03,0.480000
75%,13.100000,14.800000,264.800000,999.99,999.99,9.999000e+01,9.999990e+03,9.999990e+03,0.720000
max,36.940000,44.310000,360.000000,999.99,999.99,9.999000e+01,9.999990e+03,9.999990e+03,4.830000


Acceso a los índices, valores, columnas (las `Series` no disponen de este atributo):

In [11]:
wind.index

DatetimeIndex(['2013-09-04 00:00:00', '2013-09-04 00:10:00',
               '2013-09-04 00:20:00', '2013-09-04 00:30:00',
               '2013-09-04 00:40:00', '2013-09-04 00:50:00',
               '2013-09-04 01:00:00', '2013-09-04 01:10:00',
               '2013-09-04 01:20:00', '2013-09-04 01:30:00',
               ...
               '2015-09-03 10:30:00', '2015-09-03 10:40:00',
               '2015-09-03 10:50:00', '2015-09-03 11:00:00',
               '2015-09-03 11:10:00', '2015-09-03 11:20:00',
               '2015-09-03 11:30:00', '2015-09-03 11:40:00',
               '2015-09-03 11:50:00', '2015-09-03 12:00:00'],
              dtype='datetime64[ns]', name='Date_time', length=100886, freq=None)

In [12]:
wind.values

array([[  2.21000000e+00,   2.58000000e+00,   1.13500000e+02, ...,
          9.99999000e+03,   9.99999000e+03,   1.10000000e-01],
       [  1.69000000e+00,   2.31000000e+00,   9.99000000e+01, ...,
          9.99999000e+03,   9.99999000e+03,   3.50000000e-01],
       [  1.28000000e+00,   1.50000000e+00,   9.60000000e+01, ...,
          9.99999000e+03,   9.99999000e+03,   8.00000000e-02],
       ..., 
       [  6.66000000e+00,   8.54000000e+00,   3.03900000e+02, ...,
          9.99999000e+03,   9.99999000e+03,   5.30000000e-01],
       [  6.28000000e+00,   7.31000000e+00,   3.07100000e+02, ...,
          9.99999000e+03,   9.99999000e+03,   3.40000000e-01],
       [  6.51000000e+00,   8.31000000e+00,   3.05800000e+02, ...,
          9.99999000e+03,   9.99999000e+03,   4.80000000e-01]])

In [13]:
wind.values.shape

(100886, 9)

In [14]:
wind.columns

Index(['wspd', 'wspd_max', 'wdir', 'x1', 'x2', 'x3', 'x4', 'x5', 'wspd_std'], dtype='object')

# Eliminando/extrayendo columnas

Hay varias columnas que no nos interesan (las que hemos llamado *x1*, *x2*, *x3*, *x4* y *x5*). Vamos a eliminarlas de nuestro `DataFrame`. Lo podemos hacer de varias formas:

In [15]:
# eliminamos una columna usando la keyword 'del'
del wind['x1']
wind.head(3)

,wspd,wspd_max,wdir,x2,x3,x4,x5,wspd_std
Date_time,,,,,,,,
2013-09-04 00:00:00,2.21,2.58,113.5,999.99,99.99,9999.99,9999.99,0.11
2013-09-04 00:10:00,1.69,2.31,99.9,999.99,99.99,9999.99,9999.99,0.35
2013-09-04 00:20:00,1.28,1.50,96.0,999.99,99.99,9999.99,9999.99,0.08


In [16]:
# Extraemos una columna usando el método pop
s = wind.pop('x2')
wind.head(3)

,wspd,wspd_max,wdir,x3,x4,x5,wspd_std
Date_time,,,,,,,
2013-09-04 00:00:00,2.21,2.58,113.5,99.99,9999.99,9999.99,0.11
2013-09-04 00:10:00,1.69,2.31,99.9,99.99,9999.99,9999.99,0.35
2013-09-04 00:20:00,1.28,1.50,96.0,99.99,9999.99,9999.99,0.08


In [17]:
del wind['x3']
del wind['x4']
del wind['x5']

In [18]:
wind.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100886 entries, 2013-09-04 00:00:00 to 2015-09-03 12:00:00
Data columns (total 4 columns):
wspd        100886 non-null float64
wspd_max    100886 non-null float64
wdir        100886 non-null float64
wspd_std    100886 non-null float64
dtypes: float64(4)
memory usage: 3.8 MB


Una de las columnas, la que hemos extraído con el método `pop`, la hemos almacenado en la variable `s`, que es una `Series`:

In [19]:
type(s)

pandas.core.series.Series

In [20]:
s.head(3)

Date_time
2013-09-04 00:00:00    999.99
2013-09-04 00:10:00    999.99
2013-09-04 00:20:00    999.99
Name: x2, dtype: float64

In [21]:
s.info()

AttributeError: 'Series' object has no attribute 'info'

¿Es una TimeSeries?, i.e., ¿todos los índices son fechas?

In [22]:
# s.is_time_series deprecated
s.index.is_all_dates

True

In [23]:
s.describe()

count    100886.00
mean        999.99
std           0.00
min         999.99
25%         999.99
50%         999.99
75%         999.99
max         999.99
Name: x2, dtype: float64

In [24]:
s.dtype

dtype('float64')

In [25]:
s.values

array([ 999.99,  999.99,  999.99, ...,  999.99,  999.99,  999.99])

In [26]:
s.index

DatetimeIndex(['2013-09-04 00:00:00', '2013-09-04 00:10:00',
               '2013-09-04 00:20:00', '2013-09-04 00:30:00',
               '2013-09-04 00:40:00', '2013-09-04 00:50:00',
               '2013-09-04 01:00:00', '2013-09-04 01:10:00',
               '2013-09-04 01:20:00', '2013-09-04 01:30:00',
               ...
               '2015-09-03 10:30:00', '2015-09-03 10:40:00',
               '2015-09-03 10:50:00', '2015-09-03 11:00:00',
               '2015-09-03 11:10:00', '2015-09-03 11:20:00',
               '2015-09-03 11:30:00', '2015-09-03 11:40:00',
               '2015-09-03 11:50:00', '2015-09-03 12:00:00'],
              dtype='datetime64[ns]', name='Date_time', length=100886, freq=None)

In [27]:
s.columns

AttributeError: 'Series' object has no attribute 'columns'

# Trabajando con los índices

In [28]:
# Creamos un DataFrame
df = pd.DataFrame(np.array([['a','b','c','d','e'], [10,20,30,40,50]]).T,
                  columns = ['col1', 'col2'])
df

,col1,col2
0,a,10
1,b,20
2,c,30
3,d,40
4,e,50


Los índices se pueden reescribir en cualquier momento:

In [29]:
df.index = np.arange(1,6) * 100
df

,col1,col2
100,a,10
200,b,20
300,c,30
400,d,40
500,e,50


Podemos usar una columna para definir nuestros índices:

In [30]:
df.set_index('col1', inplace = True)
df

,col2
col1,
a,10
b,20
c,30
d,40
e,50


Podemos deshacer lo anterior usando:

In [31]:
df.reset_index(inplace = True)
df

,col1,col2
0,a,10
1,b,20
2,c,30
3,d,40
4,e,50


También podemos cambiar los nombres de las columnas:

In [32]:
df.columns = ['column1', 'column2']
df

,column1,column2
0,a,10
1,b,20
2,c,30
3,d,40
4,e,50


Podemos darle un nombre a la columna de índices (como hemos visto anteriormente):

In [33]:
df.index.name = 'indices'
df

,column1,column2
indices,,
0,a,10
1,b,20
2,c,30
3,d,40
4,e,50


# Las estructuras de datos de pandas son numpy arrays con esteroides

No olvidemos que entre bambalinas tenemos numpy arrays y pandas expone mucha de la funcionalidad de los numpy arrays directamente dentro de sus propias estructuras de datos.

Vemos, por ejemplo, qué atributos de un numpy array están disponibles directamente en una `Series` (o en un `DataFrame`):

In [34]:
numpy_attrs = dir(s.values)
series_attrs = dir(s)
for attr in numpy_attrs:
    if attr not in series_attrs:
        print('NOOOOOOOOOOOOOOOOOOOOOO', attr)
    else:
        print(attr)

T
__abs__
__add__
__and__
__array__
NOOOOOOOOOOOOOOOOOOOOOO __array_finalize__
NOOOOOOOOOOOOOOOOOOOOOO __array_interface__
__array_prepare__
__array_priority__
NOOOOOOOOOOOOOOOOOOOOOO __array_struct__
__array_wrap__
__bool__
__class__
__contains__
NOOOOOOOOOOOOOOOOOOOOOO __copy__
NOOOOOOOOOOOOOOOOOOOOOO __deepcopy__
__delattr__
__delitem__
__dir__
NOOOOOOOOOOOOOOOOOOOOOO __divmod__
__doc__
__eq__
__float__
__floordiv__
__format__
__ge__
__getattribute__
__getitem__
__gt__
__hash__
__iadd__
NOOOOOOOOOOOOOOOOOOOOOO __iand__
NOOOOOOOOOOOOOOOOOOOOOO __ifloordiv__
NOOOOOOOOOOOOOOOOOOOOOO __ilshift__
NOOOOOOOOOOOOOOOOOOOOOO __imatmul__
NOOOOOOOOOOOOOOOOOOOOOO __imod__
__imul__
NOOOOOOOOOOOOOOOOOOOOOO __index__
__init__
__int__
__invert__
NOOOOOOOOOOOOOOOOOOOOOO __ior__
__ipow__
NOOOOOOOOOOOOOOOOOOOOOO __irshift__
__isub__
__iter__
__itruediv__
NOOOOOOOOOOOOOOOOOOOOOO __ixor__
__le__
__len__
NOOOOOOOOOOOOOOOOOOOOOO __lshift__
__lt__
NOOOOOOOOOOOOOOOOOOOOOO __matmul__
__mod__
__mul__
__ne__
__

Por tanto, muchas operaciones que hacemos con un numpy array la tenemos directamente disponible con una estructura de datos de Pandas:

In [35]:
s.mean()

999.99000000000001

In [36]:
s.min()

999.99000000000001

In [37]:
s.max()

999.99000000000001

In [38]:
s[0:10].tolist()

[999.99000000000001,
 999.99000000000001,
 999.99000000000001,
 999.99000000000001,
 999.99000000000001,
 999.99000000000001,
 999.99000000000001,
 999.99000000000001,
 999.99000000000001,
 999.99000000000001]

...

# ¿Y dónde están los esteroides?

Sed pacientes!!!!!!

## 'Cosas' que están en una `Series` pero no en un numpy array

In [39]:
numpy_attrs = dir(s.values)
series_attrs = dir(s)
for attr in series_attrs:
    if attr not in numpy_attrs:
        print('NOOOOOOOOOOOOOOOOOOOOOO', attr)
    else:
        print(attr)

T
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_ALIASES
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_IALIASES
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_LEN
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_NAMES
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_NUMBERS
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_ORDERS
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_REVERSED
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_SLICEMAP
__abs__
__add__
__and__
__array__
__array_prepare__
__array_priority__
__array_wrap__
__bool__
NOOOOOOOOOOOOOOOOOOOOOO __bytes__
__class__
__contains__
__delattr__
__delitem__
NOOOOOOOOOOOOOOOOOOOOOO __dict__
__dir__
NOOOOOOOOOOOOOOOOOOOOOO __div__
__doc__
__eq__
NOOOOOOOOOOOOOOOOOOOOOO __finalize__
__float__
__floordiv__
__format__
__ge__
NOOOOOOOOOOOOOOOOOOOOOO __getattr__
__getattribute__
__getitem__
NOOOOOOOOOOOOOOOOOOOOOO __getstate__
__gt__
__hash__
__iadd__
__imul__
__init__
__int__
__invert__
__ipow__
__isub__
__iter__
__itruediv__
__le__
__len__
NOOOOOOOOOOOOOOOOOOOOOO __long__
__lt__
__mod__
NOOOOOOOOOOOOOOOOOOOOOO __module__
__mul__
__ne__
__neg__
__new__
NOOOOOOOOOOOOOOOOOOOOOO

## 'Cosas' que están en un `DataFrame` pero no en un numpy array.

In [40]:
numpy_attrs = dir(s.values)
dataframe_attrs = dir(wind)
for attr in dataframe_attrs:
    if attr not in numpy_attrs:
        print('NOOOOOOOOOOOOOOOOOOOOOO', attr)
    else:
        print(attr)

T
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_ALIASES
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_IALIASES
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_LEN
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_NAMES
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_NUMBERS
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_ORDERS
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_REVERSED
NOOOOOOOOOOOOOOOOOOOOOO _AXIS_SLICEMAP
__abs__
__add__
__and__
__array__
__array_wrap__
__bool__
NOOOOOOOOOOOOOOOOOOOOOO __bytes__
__class__
__contains__
__delattr__
__delitem__
NOOOOOOOOOOOOOOOOOOOOOO __dict__
__dir__
NOOOOOOOOOOOOOOOOOOOOOO __div__
__doc__
__eq__
NOOOOOOOOOOOOOOOOOOOOOO __finalize__
__floordiv__
__format__
__ge__
NOOOOOOOOOOOOOOOOOOOOOO __getattr__
__getattribute__
__getitem__
NOOOOOOOOOOOOOOOOOOOOOO __getstate__
__gt__
__hash__
__iadd__
__imul__
__init__
__invert__
__ipow__
__isub__
__iter__
__itruediv__
__le__
__len__
__lt__
__mod__
NOOOOOOOOOOOOOOOOOOOOOO __module__
__mul__
__ne__
__neg__
__new__
NOOOOOOOOOOOOOOOOOOOOOO __nonzero__
__or__
__pow__
__radd__
__rand__
NOOOOOOOOOOOOOOOOOOOOOO __rdiv__
__reduce_

## Ejemplos de algunas operaciones muy útiles.

Veremos estas cosas más en detalle y con ejemplos en próximos notebooks.

In [41]:
wind['wspd'].apply(lambda x: str(x) + ' m/s')

Date_time
2013-09-04 00:00:00    2.21 m/s
2013-09-04 00:10:00    1.69 m/s
2013-09-04 00:20:00    1.28 m/s
2013-09-04 00:30:00    1.94 m/s
2013-09-04 00:40:00    2.17 m/s
2013-09-04 00:50:00    2.25 m/s
2013-09-04 01:00:00    2.66 m/s
2013-09-04 01:10:00    3.55 m/s
2013-09-04 01:20:00    4.26 m/s
2013-09-04 01:30:00    4.62 m/s
2013-09-04 01:40:00     5.3 m/s
2013-09-04 01:50:00    4.68 m/s
2013-09-04 02:00:00     4.7 m/s
2013-09-04 02:10:00    4.87 m/s
2013-09-04 02:20:00    5.14 m/s
2013-09-04 02:30:00    5.54 m/s
2013-09-04 02:40:00    5.41 m/s
2013-09-04 02:50:00    5.16 m/s
2013-09-04 03:00:00    4.72 m/s
2013-09-04 03:10:00    4.56 m/s
2013-09-04 03:20:00    4.32 m/s
2013-09-04 03:30:00    4.25 m/s
2013-09-04 03:40:00    3.84 m/s
2013-09-04 03:50:00    3.95 m/s
2013-09-04 04:00:00     3.5 m/s
2013-09-04 04:10:00    3.36 m/s
2013-09-04 04:20:00     3.8 m/s
2013-09-04 04:30:00    4.12 m/s
2013-09-04 04:40:00    4.25 m/s
2013-09-04 04:50:00    4.03 m/s
                         ...  

In [42]:
wind.corr()

,wspd,wspd_max,wdir,wspd_std
wspd,1.000000,0.993587,0.125675,0.734330
wspd_max,0.993587,1.000000,0.138894,0.798218
wdir,0.125675,0.138894,1.000000,0.168828
wspd_std,0.734330,0.798218,0.168828,1.000000


In [43]:
wind.cumsum()

,wspd,wspd_max,wdir,wspd_std
Date_time,,,,
2013-09-04 00:00:00,2.21,2.58,113.5,0.11
2013-09-04 00:10:00,3.90,4.89,213.4,0.46
2013-09-04 00:20:00,5.18,6.39,309.4,0.54
2013-09-04 00:30:00,7.12,8.78,408.6,0.80
2013-09-04 00:40:00,9.29,11.45,517.0,1.03
2013-09-04 00:50:00,11.54,14.34,622.0,1.38
2013-09-04 01:00:00,14.20,17.30,736.4,1.50
2013-09-04 01:10:00,17.75,21.63,855.5,1.96
2013-09-04 01:20:00,22.01,26.13,977.6,2.04


In [44]:
wind.diff()

,wspd,wspd_max,wdir,wspd_std
Date_time,,,,
2013-09-04 00:00:00,NaN,NaN,NaN,NaN
2013-09-04 00:10:00,-0.52,-0.27,-13.6,0.24
2013-09-04 00:20:00,-0.41,-0.81,-3.9,-0.27
2013-09-04 00:30:00,0.66,0.89,3.2,0.18
2013-09-04 00:40:00,0.23,0.28,9.2,-0.03
2013-09-04 00:50:00,0.08,0.22,-3.4,0.12
2013-09-04 01:00:00,0.41,0.07,9.4,-0.23
2013-09-04 01:10:00,0.89,1.37,4.7,0.34
2013-09-04 01:20:00,0.71,0.17,3.0,-0.38


De momento, hasta que no veamos como hacer una serie de operaciones, estamos pasando por todo esto por encima. Luego veremos algunas cosas más en detalle...

Hagamos algunos ejemplos sencillos:

In [39]:
# obtened la velocidad media del viento (columna 'wspd'):


In [40]:
# obtened la mediana de la dirección del viento (columna 'wdir'):


In [41]:
# obtened el valor máximo de la diferencia entre dos 
# pasos temporales de la desviación estándar de la velocidad (columna 'wspd_std')


Otros métodos interesantes son los `rolling`:

In [47]:
pd.rolling_mean(wind, 5, center = True).head(10)

/home/tornado/miniconda3/envs/sciconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=5,center=True).mean()
  if __name__ == '__main__':


,wspd,wspd_max,wdir,wspd_std
Date_time,,,,
2013-09-04 00:00:00,NaN,NaN,NaN,NaN
2013-09-04 00:10:00,NaN,NaN,NaN,NaN
2013-09-04 00:20:00,1.858,2.290,103.40,0.206
2013-09-04 00:30:00,1.866,2.352,101.70,0.254
2013-09-04 00:40:00,2.060,2.482,104.60,0.208
2013-09-04 00:50:00,2.514,3.048,109.22,0.284
2013-09-04 01:00:00,2.978,3.470,113.80,0.248
2013-09-04 01:10:00,3.468,3.956,116.28,0.248
2013-09-04 01:20:00,4.078,4.504,120.24,0.210


Como podéis leer en el mensaje de error estas funciones `rolling_*` dejarán de existir en el futuro. En la celda de texto anterior he hablado de métodos de forma explícita porque todas las funciones `rolling_*` se han agrupado en el método `rolling`. Veamos lo anterior con la nueva forma:

In [49]:
wind.rolling(5, center = True).mean().head(10)

,wspd,wspd_max,wdir,wspd_std
Date_time,,,,
2013-09-04 00:00:00,NaN,NaN,NaN,NaN
2013-09-04 00:10:00,NaN,NaN,NaN,NaN
2013-09-04 00:20:00,1.858,2.290,103.40,0.206
2013-09-04 00:30:00,1.866,2.352,101.70,0.254
2013-09-04 00:40:00,2.060,2.482,104.60,0.208
2013-09-04 00:50:00,2.514,3.048,109.22,0.284
2013-09-04 01:00:00,2.978,3.470,113.80,0.248
2013-09-04 01:10:00,3.468,3.956,116.28,0.248
2013-09-04 01:20:00,4.078,4.504,120.24,0.210


Otras cosas a las que les podéis echar un ojo en un `DataFrame` (cambiad `DataFrame` por `Series` o cualquier otra estructura de datos de vuestro interés):

In [57]:
import inspect
info = inspect.getmembers(wind, predicate=inspect.ismethod)

for stuff in info:
    print(stuff[0])

__abs__
__add__
__and__
__array__
__array_wrap__
__bool__
__bytes__
__contains__
__delitem__
__dir__
__div__
__eq__
__finalize__
__floordiv__
__ge__
__getattr__
__getitem__
__getstate__
__gt__
__hash__
__iadd__
__imul__
__init__
__invert__
__ipow__
__isub__
__iter__
__itruediv__
__le__
__len__
__lt__
__mod__
__mul__
__ne__
__neg__
__nonzero__
__or__
__pow__
__radd__
__rand__
__rdiv__
__repr__
__rfloordiv__
__rmod__
__rmul__
__ror__
__round__
__rpow__
__rsub__
__rtruediv__
__rxor__
__setattr__
__setitem__
__setstate__
__sizeof__
__str__
__sub__
__truediv__
__unicode__
__xor__
_add_numeric_operations
_add_series_only_operations
_add_series_or_dataframe_operations
_agg_by_level
_align_frame
_align_series
_apply_broadcast
_apply_empty_result
_apply_raw
_apply_standard
_box_col_values
_box_item_values
_check_inplace_setting
_check_is_chained_assignment_possible
_check_percentile
_check_setitem_copy
_clear_item_cache
_combine_const
_combine_frame
_combine_match_columns
_combine_match_index
_

# Trabajando con datos perdidos

Es normal que en nuestros datos tengamos valores perdidos.

In [58]:
index = pd.date_range('2000/01/01', freq = '12H', periods = 10)
index = index.append(pd.date_range('2000/01/10', freq = '1D', periods = 3))
df = pd.DataFrame(np.random.randint(1, 100, size = (13, 3)), 
                  index = index, columns = ['col1', 'col2', 'col3'])
df

,col1,col2,col3
2000-01-01 00:00:00,17,68,65
2000-01-01 12:00:00,98,49,78
2000-01-02 00:00:00,75,82,21
2000-01-02 12:00:00,62,88,96
2000-01-03 00:00:00,53,44,64
2000-01-03 12:00:00,98,7,23
2000-01-04 00:00:00,72,77,40
2000-01-04 12:00:00,98,96,83
2000-01-05 00:00:00,88,11,60
2000-01-05 12:00:00,75,28,63


In [59]:
# Vamos a rellenar algunos valores con NaN
df[df > 70] = np.nan
df

,col1,col2,col3
2000-01-01 00:00:00,17.0,68.0,65.0
2000-01-01 12:00:00,NaN,49.0,NaN
2000-01-02 00:00:00,NaN,NaN,21.0
2000-01-02 12:00:00,62.0,NaN,NaN
2000-01-03 00:00:00,53.0,44.0,64.0
2000-01-03 12:00:00,NaN,7.0,23.0
2000-01-04 00:00:00,NaN,NaN,40.0
2000-01-04 12:00:00,NaN,NaN,NaN
2000-01-05 00:00:00,NaN,11.0,60.0
2000-01-05 12:00:00,NaN,28.0,63.0


A diferencia de lo que sucede con un numpy array, en pandas, las operaciones con `NaN` ignoran los mismos. Veamos esto en acción:

In [60]:
df['col1'].sum()

171.0

In [61]:
df['col1'].values.sum()

nan

Podemos detectar los valores nulos (`NaN`) usando `isnull`:

In [62]:
df.isnull()

,col1,col2,col3
2000-01-01 00:00:00,False,False,False
2000-01-01 12:00:00,True,False,True
2000-01-02 00:00:00,True,True,False
2000-01-02 12:00:00,False,True,True
2000-01-03 00:00:00,False,False,False
2000-01-03 12:00:00,True,False,False
2000-01-04 00:00:00,True,True,False
2000-01-04 12:00:00,True,True,True
2000-01-05 00:00:00,True,False,False
2000-01-05 12:00:00,True,False,False


O los no nulos usando `notnull`:

In [63]:
df.notnull()

,col1,col2,col3
2000-01-01 00:00:00,True,True,True
2000-01-01 12:00:00,False,True,False
2000-01-02 00:00:00,False,False,True
2000-01-02 12:00:00,True,False,False
2000-01-03 00:00:00,True,True,True
2000-01-03 12:00:00,False,True,True
2000-01-04 00:00:00,False,False,True
2000-01-04 12:00:00,False,False,False
2000-01-05 00:00:00,False,True,True
2000-01-05 12:00:00,False,True,True


Vemos que tenemos valores `NaN`. Estos los podemos rellenar usando `ffill` o `bfill` (similar a `fillna(method = 'ffill')` y a `fillna(method = 'bfill')`, respectivamente):

In [64]:
# Recordemos como era nuestro DataFrame
df

,col1,col2,col3
2000-01-01 00:00:00,17.0,68.0,65.0
2000-01-01 12:00:00,NaN,49.0,NaN
2000-01-02 00:00:00,NaN,NaN,21.0
2000-01-02 12:00:00,62.0,NaN,NaN
2000-01-03 00:00:00,53.0,44.0,64.0
2000-01-03 12:00:00,NaN,7.0,23.0
2000-01-04 00:00:00,NaN,NaN,40.0
2000-01-04 12:00:00,NaN,NaN,NaN
2000-01-05 00:00:00,NaN,11.0,60.0
2000-01-05 12:00:00,NaN,28.0,63.0


In [65]:
df.ffill()

,col1,col2,col3
2000-01-01 00:00:00,17.0,68.0,65.0
2000-01-01 12:00:00,17.0,49.0,65.0
2000-01-02 00:00:00,17.0,49.0,21.0
2000-01-02 12:00:00,62.0,49.0,21.0
2000-01-03 00:00:00,53.0,44.0,64.0
2000-01-03 12:00:00,53.0,7.0,23.0
2000-01-04 00:00:00,53.0,7.0,40.0
2000-01-04 12:00:00,53.0,7.0,40.0
2000-01-05 00:00:00,53.0,11.0,60.0
2000-01-05 12:00:00,53.0,28.0,63.0


In [66]:
df.bfill()

,col1,col2,col3
2000-01-01 00:00:00,17.0,68.0,65.0
2000-01-01 12:00:00,62.0,49.0,21.0
2000-01-02 00:00:00,62.0,44.0,21.0
2000-01-02 12:00:00,62.0,44.0,64.0
2000-01-03 00:00:00,53.0,44.0,64.0
2000-01-03 12:00:00,26.0,7.0,23.0
2000-01-04 00:00:00,26.0,11.0,40.0
2000-01-04 12:00:00,26.0,11.0,60.0
2000-01-05 00:00:00,26.0,11.0,60.0
2000-01-05 12:00:00,26.0,28.0,63.0


In [67]:
df.fillna(value = 'Kiko')

,col1,col2,col3
2000-01-01 00:00:00,17,68,65
2000-01-01 12:00:00,Kiko,49,Kiko
2000-01-02 00:00:00,Kiko,Kiko,21
2000-01-02 12:00:00,62,Kiko,Kiko
2000-01-03 00:00:00,53,44,64
2000-01-03 12:00:00,Kiko,7,23
2000-01-04 00:00:00,Kiko,Kiko,40
2000-01-04 12:00:00,Kiko,Kiko,Kiko
2000-01-05 00:00:00,Kiko,11,60
2000-01-05 12:00:00,Kiko,28,63


Creamos un nuevo `DataFrame` con un frecuencia de 12H en los índices.

In [68]:
df = pd.DataFrame(np.random.randint(1, 100, size = (15, 3)), 
                  index = pd.date_range('2015/01/01', freq = '12H', periods = 15))
df

,0,1,2
2015-01-01 00:00:00,51,9,62
2015-01-01 12:00:00,25,71,29
2015-01-02 00:00:00,60,50,54
2015-01-02 12:00:00,90,78,14
2015-01-03 00:00:00,84,75,49
2015-01-03 12:00:00,6,89,33
2015-01-04 00:00:00,17,91,90
2015-01-04 12:00:00,97,48,29
2015-01-05 00:00:00,44,33,22
2015-01-05 12:00:00,71,2,68


In [69]:
df[df > 70] = 'Kiko'
df

,0,1,2
2015-01-01 00:00:00,51,9,62
2015-01-01 12:00:00,25,Kiko,29
2015-01-02 00:00:00,60,50,54
2015-01-02 12:00:00,Kiko,Kiko,14
2015-01-03 00:00:00,Kiko,Kiko,49
2015-01-03 12:00:00,6,Kiko,33
2015-01-04 00:00:00,17,Kiko,Kiko
2015-01-04 12:00:00,Kiko,48,29
2015-01-05 00:00:00,44,33,22
2015-01-05 12:00:00,Kiko,2,68


Podemos eliminar las filas o columnas que tengan algún valor `NaN`, todos loa valores `NaN`,...

In [70]:
df[df == 'Kiko'] = np.nan
df

,0,1,2
2015-01-01 00:00:00,51,9,62
2015-01-01 12:00:00,25,NaN,29
2015-01-02 00:00:00,60,50,54
2015-01-02 12:00:00,NaN,NaN,14
2015-01-03 00:00:00,NaN,NaN,49
2015-01-03 12:00:00,6,NaN,33
2015-01-04 00:00:00,17,NaN,NaN
2015-01-04 12:00:00,NaN,48,29
2015-01-05 00:00:00,44,33,22
2015-01-05 12:00:00,NaN,2,68


In [71]:
# Eliminamos las filas donde algún valor de la fila sea NaN
df.dropna(axis = 'rows') # axis = 0 sería equivalente a axis = 'rows'

,0,1,2
2015-01-01 00:00:00,51,9,62
2015-01-02 00:00:00,60,50,54
2015-01-05 00:00:00,44,33,22
2015-01-07 12:00:00,17,54,58


In [72]:
# Eliminamos las filas donde todos los valores de la fila sean NaN
df.iloc[2, :] = np.nan
df.dropna(axis = 'rows', how = 'all') # axis = 0 sería equivalente a axis = 'rows'

,0,1,2
2015-01-01 00:00:00,51,9,62
2015-01-01 12:00:00,25,NaN,29
2015-01-02 12:00:00,NaN,NaN,14
2015-01-03 00:00:00,NaN,NaN,49
2015-01-03 12:00:00,6,NaN,33
2015-01-04 00:00:00,17,NaN,NaN
2015-01-04 12:00:00,NaN,48,29
2015-01-05 00:00:00,44,33,22
2015-01-05 12:00:00,NaN,2,68
2015-01-06 00:00:00,62,21,NaN


In [73]:
# Eliminamos las columnas donde algún valor de la columna sea NaN
df.dropna(axis = 'columns', how = 'any') # axis = 1 sería equivalente a axis = 'columns'
                                         # how = 'any' es el valor por defecto por lo que no sería necesario añadirlo

""
2015-01-01 00:00:00
2015-01-01 12:00:00
2015-01-02 00:00:00
2015-01-02 12:00:00
2015-01-03 00:00:00
2015-01-03 12:00:00
2015-01-04 00:00:00
2015-01-04 12:00:00
2015-01-05 00:00:00
2015-01-05 12:00:00


In [74]:
# Vamos a añadir una columna que no tenga NaNs y vamos a hacer a misma operación
df['col4'] = 9999
df.dropna(axis = 'columns', how = 'any')

,col4
2015-01-01 00:00:00,9999
2015-01-01 12:00:00,9999
2015-01-02 00:00:00,9999
2015-01-02 12:00:00,9999
2015-01-03 00:00:00,9999
2015-01-03 12:00:00,9999
2015-01-04 00:00:00,9999
2015-01-04 12:00:00,9999
2015-01-05 00:00:00,9999
2015-01-05 12:00:00,9999


In [75]:
# Ahora vamos a añadir una columna donde todos los valores sean NaN
df['col5'] = np.nan
df.dropna(axis = 'columns', how = 'all')

,0,1,2,col4
2015-01-01 00:00:00,51,9,62,9999
2015-01-01 12:00:00,25,NaN,29,9999
2015-01-02 00:00:00,NaN,NaN,NaN,9999
2015-01-02 12:00:00,NaN,NaN,14,9999
2015-01-03 00:00:00,NaN,NaN,49,9999
2015-01-03 12:00:00,6,NaN,33,9999
2015-01-04 00:00:00,17,NaN,NaN,9999
2015-01-04 12:00:00,NaN,48,29,9999
2015-01-05 00:00:00,44,33,22,9999
2015-01-05 12:00:00,NaN,2,68,9999


Tambien podemos rellenar los valores NaN usando `interpolate`:

In [77]:
df.interpolate()

,0,1,2,col4,col5
2015-01-01 00:00:00,51,9,62,9999,NaN
2015-01-01 12:00:00,25,NaN,29,9999,NaN
2015-01-02 00:00:00,NaN,NaN,NaN,9999,NaN
2015-01-02 12:00:00,NaN,NaN,14,9999,NaN
2015-01-03 00:00:00,NaN,NaN,49,9999,NaN
2015-01-03 12:00:00,6,NaN,33,9999,NaN
2015-01-04 00:00:00,17,NaN,NaN,9999,NaN
2015-01-04 12:00:00,NaN,48,29,9999,NaN
2015-01-05 00:00:00,44,33,22,9999,NaN
2015-01-05 12:00:00,NaN,2,68,9999,NaN


Pero que ha pasado!!! ¿Por qué no ha interpolado?

Veamos como son las columnas

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15 entries, 2015-01-01 00:00:00 to 2015-01-08 00:00:00
Freq: 12H
Data columns (total 5 columns):
0       9 non-null object
1       8 non-null object
2       10 non-null object
col4    15 non-null int64
col5    0 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 720.0+ bytes


Vemos que las columnas `0`, `1` y `2` son del tipo `object` y no son un número. Por otro lado, en la columna `col4` no hay nada que interpolar puesto que no hay valores perdidos. Por último, la columna `col5` son todo `NaN` por lo que no se puede inventar la información. Transformemos las tres primeras columnas e interpolemos:

In [79]:
df[[0, 1, 2]] = df[[0, 1, 2]].astype(np.float)

In [80]:
df.interpolate()

,0,1,2,col4,col5
2015-01-01 00:00:00,51.00,9.000000,62.0,9999,NaN
2015-01-01 12:00:00,25.00,14.571429,29.0,9999,NaN
2015-01-02 00:00:00,20.25,20.142857,21.5,9999,NaN
2015-01-02 12:00:00,15.50,25.714286,14.0,9999,NaN
2015-01-03 00:00:00,10.75,31.285714,49.0,9999,NaN
2015-01-03 12:00:00,6.00,36.857143,33.0,9999,NaN
2015-01-04 00:00:00,17.00,42.428571,31.0,9999,NaN
2015-01-04 12:00:00,30.50,48.000000,29.0,9999,NaN
2015-01-05 00:00:00,44.00,33.000000,22.0,9999,NaN
2015-01-05 12:00:00,53.00,2.000000,68.0,9999,NaN


In [ ]:
# Mirad la ayuda del método interpolate para aprender mejor como usarlo
